# Combine Postcode and SA2

In [1]:
import pandas as pd

In [2]:
postcode = pd.read_csv('../data/landing/australian_postcodes.csv')
filtered_vic = postcode[postcode['state'] == 'VIC']
filtered_vic.head()

,id,postcode,locality,state,long,lat,dc,type,status,sa3,...,MMM_2019,ced,altitude,chargezone,phn_code,phn_name,lgaregion,lgacode,electorate,electoraterating
6185,4746,3000,MELBOURNE,VIC,144.956776,-37.817403,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,...,1.0,Melbourne,15.244751,V1,PHN201,North Western Melbourne,Melbourne,24600.0,Melbourne,Inner Metropolitan
6186,4747,3001,MELBOURNE,VIC,144.956776,-37.817403,CITY MAIL PROCESSING CENTRE,Post Office Boxes,Updated 25-Mar-2020 SA3,20605.0,...,1.0,Melbourne,15.244751,V1,PHN203,NaN,Melbourne,24600.0,Maribyrnong,NaN
6187,4748,3002,EAST MELBOURNE,VIC,144.982207,-37.818517,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,...,1.0,Melbourne,14.315022,V1,PHN201,North Western Melbourne,Yarra,27350.0,Melbourne,Inner Metropolitan
6188,4749,3003,WEST MELBOURNE,VIC,144.949592,-37.810871,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20604.0,...,1.0,Melbourne,15.316063,V1,PHN201,North Western Melbourne,Melbourne,24600.0,Melbourne,Inner Metropolitan
6189,4750,3004,MELBOURNE,VIC,144.970161,-37.844246,CITY DELIVERY CENTRE,Delivery Area,Updated 6-Feb-2020,20605.0,...,1.0,Macnamara,-3492.000000,V1,PHN203,South Eastern Melbourne,Port Phillip,25900.0,Macnamara,Inner Metropolitan


In [3]:
postcode_to_sa2 = filtered_vic[['postcode', 'SA2_MAINCODE_2016']]
postcode_to_sa2.rename(columns={'SA2_MAINCODE_2016': 'SA2'}, inplace=True)
postcode_to_sa2 = postcode_to_sa2[postcode_to_sa2['SA2'].notnull()]
postcode_to_sa2['SA2'] = postcode_to_sa2['SA2'].astype(int)

# Remove duplicates from postcode_to_sa2 to ensure a one-to-one mapping
postcode_to_sa2 = postcode_to_sa2.drop_duplicates(subset='postcode', keep='first')


# Filter the DataFrame to include only rows with postcodes between 3000 and 4000
filtered_sa2_postcode = postcode_to_sa2[
    (postcode_to_sa2['postcode'] >= 3000) & (postcode_to_sa2['postcode'] <= 4000)
    ]

# Reset the index if needed
filtered_sa2_postcode.reset_index(drop=True, inplace=True)
filtered_sa2_postcode


/tmp/ipykernel_17600/3912623233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  postcode_to_sa2.rename(columns={'SA2_MAINCODE_2016': 'SA2'}, inplace=True)


,postcode,SA2
0,3000,206041122
1,3001,206041122
2,3002,206041119
3,3003,206041127
4,3004,206041126
...,...,...
716,3990,205031093
717,3991,205031093
718,3992,205031093
719,3995,205031093


In [4]:
# Save the DataFrame to a CSV file
filtered_sa2_postcode.to_csv("../data/raw/Processed_SA2_Postcode.csv", index=False)

# Merge with Rental Data

In [5]:
df = pd.read_csv('../data/raw/domain_outliers_removed.csv')

# Merge df and postcode_to_sa2
merged_df = pd.merge(df, filtered_sa2_postcode, on='postcode', how='left')

merged_df.to_csv("../data/curated/domain.csv", index=False)

merged_df.head()

,name,coordinates,price,bed,bath,park_num,postcode,Latitude,Longitude,SA2
0,2010/7 Katherine Place Melbourne VIC 3000,"[-37.8201858, 144.9564628]",675.0,2.0,2.0,0,3000,-37.820186,144.956463,206041122
1,1308/68 La Trobe Street Melbourne VIC 3000,"[-37.8083648, 144.9667899]",570.0,2.0,1.0,1,3000,-37.808365,144.966790,206041122
2,1307/268 Flinders Street Melbourne VIC 3000,"[-37.8178134, 144.9655688]",400.0,1.0,1.0,0,3000,-37.817813,144.965569,206041122
3,407/62-68 Hayward Lane Melbourne VIC 3000,"[-37.8089779, 144.9668783]",350.0,0.0,1.0,0,3000,-37.808978,144.966878,206041122
4,403/39 Queen Street Melbourne VIC 3000,"[-37.817888, 144.9617372]",450.0,0.0,1.0,0,3000,-37.817888,144.961737,206041122
